# Create Traning Data for GEN2 Finetuning 
We create a new txt file "data/md/mixed_gen2/mix2_train_combined.txt" approx the same size as "data/hd/combined0/combined0/train_combined0.txt" 
of which 1/2 or 50% is human data from "data/hd/combined2/train_combined2.txt" and 1/2 or 50% is synthetic data from GEN1's, "data/sd/gen1/gen1_sd.txt".

In [1]:
import random
import os

def read_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read().strip().split('\n\n')
    return data

def write_data(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write('\n\n'.join(data))

def get_file_size(filename):
    """Returns the size of the file in bytes."""
    return os.path.getsize(filename)

# Load data
human_data = read_data('./data/hd/combined2/train_combined2.txt')
synthetic_data = read_data('./data/sd/gen1/gen1_sd.txt')

# Combine and shuffle
combined_data = human_data + synthetic_data
random.shuffle(combined_data)

# Write combined data to a new file
write_data(combined_data, './data/md/mixed_gen2/mix2_train_combined.txt')
print("Data has been successfully combined and written to mix1_train_combined.txt.")


Data has been successfully combined and written to mix1_train_combined.txt.


Check that the file size of mix2_train_combined.txt is about the same as in the original dataset for gen0 "data/hd/combined0/train_combined0.txt".

In [2]:
print("Original GEN0 HD Size: ", get_file_size("data/hd/combined0/train_combined0.txt"))
print("Training Dataset for GEN1 Size: ", get_file_size("data/md/mixed_gen2/mix2_train_combined.txt"))

Original GEN0 HD Size:  307558317
Training Dataset for GEN1 Size:  305696540


# Finetuning

Finetune the model GEN2 for 5 epochs with the combined txt file containing 1/2 real data and 1/2 synthetic data "data/md/mixed_gen2/mix2_train_combined.txt".  
The global_step parameter in "models/distilgpt2-finetuned_gen0_75/trainer_state.json" and "models/distilgpt2-finetuned_gen0_75/checkpoint-10755/trainer_state.json" is originally "global_step": 10755, but in order to force run_clm.py to  
train the model from the last checkpoint with the new dataset we set this parameter to 0 to force it to learn from the entirety of the new dataset. 

In [3]:
!deepspeed run_clm.py \
    --model_name_or_path distilgpt2 \
    --train_file data/md/mixed_gen2/mix2_train_combined.txt \
    --validation_file data/hd/initial_combined/valid_combined.txt \
    --do_train \
    --do_eval \
    --output_dir ./models/distilgpt2-finetuned_gen2_50 \
    --num_train_epochs 5 \
    --save_strategy epoch \
    --learning_rate 5e-5 \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --deepspeed ds_config.json \
    --resume_from_checkpoint ./models/distilgpt2-finetuned_gen1_75/checkpoint-10755


[2024-04-21 13:38:03,032] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-04-21 13:38:03,656] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2024-04-21 13:38:03,669] [INFO] [runner.py:568:main] cmd = /home/vasi/Documents/BA_Thesis_Experiment/.venv/bin/python -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMCwgMV19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None run_clm.py --model_name_or_path distilgpt2 --train_file data/md/mixed_gen2/mix2_train_combined.txt --validation_file data/hd/initial_combined/valid_combined.txt --do_train --do_eval --output_dir ./models/distilgpt2-finetuned_gen2_50 --num_train_epochs 5 --save_strategy epoch --learning_rate 5e-5 --per_device_train_batch_size 4 --gradient_accumulation_steps 4 --deepspeed ds_config.json --resume_from_checkpoint ./models/distilgpt2-finetuned_gen1_75/checkpoint-10755
[

# Inference
### Step 3
Let the model generate 100 stories from a ChatGPT4-generated prompts and write them to an output file.

In [4]:
with open("data/sd/eval_prompts/eval_prompts.txt") as pfile:
    prompts = pfile.readlines()

print(prompts[3].split('.')[1][1:]+".")

Deep in the Amazon rainforest, a team of scientists discovers a hidden tribe with a secret that could change the course of history.


In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "./models/distilgpt2-finetuned_gen2_50/checkpoint-10325" 
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


for prompt in prompts:
    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")


    # Generate text
    output_sequences = model.generate(
        input_ids=inputs,
        attention_mask=None,
        max_length=500,  # determines the maximum length of the generated text
        temperature=0.7,  # controls randomness: lower values make text less random
        top_k=50,  # the K most likely next words are considered for each step
        top_p=0.9,  # only the most probable tokens with probabilities that add up to top_p are considered for each step
        repetition_penalty=1.2,  # penalty applied to repeated words
        do_sample=True,  # set to True to return diverse samples
        num_return_sequences=1,  # number of independently computed samples to generate
        pad_token_id=tokenizer.eos_token_id,
    )

    # Decode the output sequences to get the generated text
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

    #print(generated_text)
    import re

    # Define regex pattern for impurities
    #pattern = r"(<newline>|<newline \d+ :>|<newline\*>|\[.*?\]|“|”|``|''|--|__________________________________________________________________|\*)"

    # Remove the prompt (first sentence) by finding the first period followed by a space or end of text
    text_without_prompt = re.split(r'\.\s*[”]*\s*(?=[A-Z])', generated_text, 1)[1] if '.' in generated_text else generated_text


    # Regex to remove specified impurities
    #cleaned_text = re.sub(r"\<[^\>]*\>|\[WP\]|\-\-", "", text_without_prompt)
    #cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Remove extra spaces and strip leading/trailing spaces
    # Apply regex to remove impurities
    #cleaned_text = re.sub(pattern, "", cleaned_text)

    print(text_without_prompt)

    with open("./outputs/gen2/stories2.txt", "a") as f:
        f.write(text_without_prompt.split("\n")[1] + "\n\n")

/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In a post-apocalyptic world where nature has reclaimed the cities, a group of survivors seeks refuge in an abandoned library.
`` There's nothing to do. ''   `` It won't work, you know? What if I get caught and killed by someone else here on earth instead of just being there for me like that... but then again, they would have no idea how long it took before anyone noticed them or even did anything about it anyway! How could something be so stupid as to make people forget what happened last time we were born.. And who knows when things will end up different than this one thing is left behind with us forever forgotten - The only difference between our two lives are those we both love now because each other loves her more every night all alone ; she was always my favorite girl after school -- ever since i grew older he had gotten old fast enough to notice him constantly coming out of bed while crying over his mommy's little pink toy toys which eventually became mine too. So many memories f

# Evaluation
### Step 5
Evaluate each story in the stories output file on all metrics and write values to df with last line being the average values of each column.
Metrics represent the model's average for all stories it generated.

In [6]:
from metrics.LexicalDiversity.lexical_diversity import *
from metrics.SemanticDiversity.sementic_diversity import *
from metrics.SyntacticDiversity.syntactic_diversity import *
from nltk.tokenize import sent_tokenize
import pandas as pd
import spacy

# Define the column names
columns = ["Distinct-2", "Distinct-3", "Self-BLEU", "OV-TTR", "MS-TTR", "S-DIV-AV", "S-DIV-C", "SYN-DIV"]

# Create an empty DataFrame with these columns
df_eval_gen1 = pd.DataFrame(columns=columns)

# Load a spaCy model for dependency parsing
nlp = spacy.load("en_core_web_sm")

with open("./outputs/gen2/stories2.txt", 'r') as f:
    stories = f.readlines()

for story in stories:
    if story != '\n':
        #print(story)
        # Tokenize the text into sentences
        sentences = sent_tokenize(story)
        graphs = construct_dependency_graphs(sentences)
    

        # Tokenize the text into sentences
        sentences = sent_tokenize(story)

        

        graphs = construct_dependency_graphs(sentences)

        # Example usage: Adding a new row of data to the DataFrame
        new_data = {
            "Distinct-2": calculate_distinct_n(story, 2),
            "Distinct-3": calculate_distinct_n(story, 3),
            "Self-BLEU": 1-calculate_self_bleu(sentences),
            "OV-TTR": calculate_ttr(story, truncate_length=300),
            "MS-TTR": calculate_mean_segmental_ttr(story, segment_size=50),
            "S-DIV-AV": calculate_semantic_diversity(sentences, 'average'),
            "S-DIV-C": calculate_semantic_diversity(sentences, 'centroid'),
            "SYN-DIV": calculate_syntactic_diversity(graphs)
        }

        # Convert new_data dictionary to a DataFrame
        new_row_df = pd.DataFrame([new_data])

        # Concatenate the new row DataFrame to the original DataFrame
        df_eval_gen1 = pd.concat([df_eval_gen1, new_row_df], ignore_index=True)
        

# Calculate the mean for each column and append as a new row
averages = df_eval_gen1.mean().to_dict()
averages = {key: [value] for key, value in averages.items()}  # Convert each mean value into a list
average_df = pd.DataFrame(averages)  # Create a DataFrame for the averages
average_df.index = ['Average']  # Label the index as 'Average'

# Append the average row to the original DataFrame
df = pd.concat([df_eval_gen1, average_df])

# Specify the file path and name
file_path = './outputs/gen2/eval_table_gen2.csv'

# Write the DataFrame to a CSV file
df.to_csv(file_path, index=False)  # Set index=False to not include row indices in the file

print(f"Data has been written to {file_path}")
# Print the last row (average values)
print("Average values for each metric:")
print(df.iloc[-1])



/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram over

Data has been written to ./outputs/gen2/eval_table_gen2.csv
Average values for each metric:
Distinct-2    0.981948
Distinct-3    0.990779
Self-BLEU     0.999904
OV-TTR        0.854711
MS-TTR        0.936705
S-DIV-AV      0.818110
S-DIV-C       0.485749
SYN-DIV       0.818231
Name: Average, dtype: float64


# Generate Synthetic Data
### Step 6
Contribute to the synthetic dataset by producing stories from the finetuned model.
We use 75% of the original prompt data as our prompt list.

In [2]:
def count_entries(filepath):
    """Counts the number of double-newline-separated entries in a file."""
    with open(filepath, 'r', encoding='utf-8') as file:
        content = file.read().strip()
    return len(content.split('\n\n'))

total_entries = count_entries("./data/hd/combined0/train_combined0.txt")
print("Total entries in GEN0 Dataset with real data: ", total_entries)

sd_entries_count= 3 * (total_entries/4)
print("75% \of total entries = ", sd_entries_count)

Total entries in GEN0 Dataset with real data:  109040
75% \of total entries =  81780.0


In [3]:
import random
from transformers import pipeline, set_seed, GPT2LMHeadModel, GPT2Tokenizer
import os

prompts = []
prompt_files = ["train", "test"]
for name in prompt_files:
    # Path to the file with prompts
    file_path = './data/hd/prepro/'+name+'.wp_source'

    # Read prompts from the file removing the initials [ XX ]
    with open(file_path, 'r', encoding='utf-8') as file:
        prompts += ([line.strip()[7:] for line in file.readlines() if line.strip()])

#print(prompts[0:10])
# Randomly select 75% of the prompts
sample_size = 3 * (len(prompts) // 4)  # 75%
selected_prompts = random.sample(prompts, int(sd_entries_count))

print(len(selected_prompts))
print(selected_prompts[0:10])


81780
['After days scrolling on reddit , you reach the end Never Ending Reddit . There is a single button .', 'It is time for a new Revelation . Last time it went quite well ; incarnating his Son into a Jewish carpenter , He converted a good part of the world to Christianity . This time the choice is harder so God asks for advice from his angel consultants . Who would be best suited for the Prophet part ?', "Life is actually the most popular video game of all time . Famous figures are played by moderators who ensure events play out the way they should . You 're one of the worst players , until you start realizing you 're in the game . Suddenly , memories of past lives give you a massive advantage .", "You 're a common goblin who has , against all odds , slain the hero of the story .", "You 've just survived a shipwreck and pull yourself onto a small island . From the shade of a palm tree a man steps out and says `` Well , well , well . We meet again . But this time the advantage is min

In [4]:
import os

def get_file_size(filename):
    """Returns the size of the file in bytes."""
    return os.path.getsize(filename)

gen0_data_filename = './data/hd/combined0/train_combined0.txt'
gen0_data_size = get_file_size(gen0_data_filename)
print(f"The size of '{gen0_data_filename}' is {gen0_data_size} bytes.")
# 75% of gen0_data_size
gen2_sd_target_size = 3 * (gen0_data_size // 4)
print("Target size (75% of GEN0) for SD File for GEN2: ", gen2_sd_target_size)


The size of './data/hd/combined0/train_combined0.txt' is 307558317 bytes.
Target size (75% of GEN0) for SD File for GEN2:  230668737


In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import re

model_name = "./models/distilgpt2-finetuned_gen2_50/checkpoint-10325"
tokenizer = GPT2Tokenizer.from_pretrained(model_name, padding_side='left')
model = GPT2LMHeadModel.from_pretrained(model_name)

# Define the device based on CUDA availability
device = "cuda" if torch.cuda.is_available() else "cpu"


if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

if torch.cuda.is_available():
    model = torch.nn.DataParallel(model)
    model.cuda()
else:
    model.to("cpu")

batch_size = 64

def generate_text_batch(prompts):
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move all tensors to the right device
    outputs = model.module.generate(
        **inputs, 
        max_length=500, 
        num_return_sequences=1, 
        temperature=0.7,  # More randomness
        repetition_penalty=1.2,  # Increase penalty to reduce repetitions
        top_k=50, 
        top_p=0.9
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

output_synth_data = './data/sd/gen2/gen2_sd.txt'

try:
    with open(output_synth_data, 'a', encoding='utf-8') as file:
        for i in range(0, len(selected_prompts), batch_size):
            batch_prompts = selected_prompts[i:i + batch_size]
            print(f"Generating text for batch {i//batch_size+1}/{len(selected_prompts)//batch_size}")
            generated_texts = generate_text_batch(batch_prompts)
            
            for prompt, generated_text in zip(batch_prompts, generated_texts):
                prompt_length = len(tokenizer.encode(prompt))
                #print(prompt)
                # Remove the prompt by slicing the tokens to skip the prompt length
                generated_text_tokens = tokenizer.encode(generated_text)[prompt_length:]
                clean_generated_text = tokenizer.decode(generated_text_tokens, skip_special_tokens=True)

                # Remove leading and ending spaces and special characters
                clean_generated_text = re.sub(r'^[\s\WP]+', '', clean_generated_text)
                clean_generated_text = re.sub(r'^[\s\W]+|[\s\W]+$', '', clean_generated_text)


                output_text = f"{prompt}\n{clean_generated_text}\n\n"
                #print(output_text)
                if get_file_size(output_synth_data) < gen2_sd_target_size:
                    file.write(output_text)
                else:
                    print("Target size reached!")
                    break
    print("Finished generating stories.")
except Exception as e:
    print(f"An error occurred: {e}")

/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generating text for batch 1/1277


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generating text for batch 2/1277


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Target size reached!
Generating text for batch 968/1277


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Target size reached!
Generating text for batch 969/1277


KeyboardInterrupt: 

In [11]:
# check total size of generated sd file
print("Target size for GEN2 sd corresponding to 3/4 of original HD data file: ", gen2_sd_target_size)
print("Actual size of GEN2's generated SD file: ", get_file_size(output_synth_data))

Target size for GEN2 sd corresponding to 3/4 of original HD data file:  230668737
Actual size of GEN2's generated SD file:  179772498
